In [1]:
import pandas as pd

In [2]:
seasons = [str(season) for season in range(2016,2025)]

In [3]:
nfl_df = pd.read_csv('nfl_betting.csv')

In [35]:
nfl_df

,Season,Team,Week,Day,Date,Result,OT,Home,Opp,Pts_F,...,Opp_Pnt,Opp_Pnt_Yds,Opp_3DConv,Opp_3DAtt,Opp_4DConv,Opp_4DAtt,Opp_ToP,G,Spread,Total
0,2016,CRD,1,Sun,September 11,L,0,1,NWE,21.0,...,2.0,89.0,10.0,16.0,0.0,0.0,33:59,1,-9.0,44.5
1,2016,CRD,2,Sun,September 18,W,0,1,TAM,40.0,...,6.0,282.0,5.0,13.0,0.0,1.0,30:57,2,-7.5,49.5
2,2016,CRD,3,Sun,September 25,L,0,0,BUF,18.0,...,5.0,178.0,4.0,13.0,0.0,0.0,26:00,3,-5.0,48.0
3,2016,CRD,4,Sun,October 2,L,0,1,RAM,13.0,...,6.0,277.0,6.0,14.0,0.0,1.0,27:51,4,-10.0,43.5
4,2016,CRD,5,Thu,October 6,W,0,0,SFO,33.0,...,7.0,349.0,4.0,14.0,1.0,1.0,28:16,5,-3.5,42.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4305,2024,WAS,2,Sun,September 15,W,0,1,NYG,21.0,...,2.0,87.0,4.0,8.0,1.0,2.0,22:28,2,-1.0,43.0
4306,2024,WAS,3,Mon,September 23,W,0,0,CIN,38.0,...,0.0,0.0,6.0,10.0,1.0,1.0,26:54,3,7.5,46.5
4307,2024,WAS,4,Sun,September 29,W,0,0,CRD,42.0,...,4.0,192.0,4.0,11.0,2.0,3.0,26:38,4,3.5,48.5
4308,2024,WAS,5,Sun,October 6,W,0,1,CLE,34.0,...,7.0,361.0,1.0,13.0,2.0,3.0,28:42,5,-3.0,43.5


In [42]:
#finding trends in data
#finding how many games with certain betting totals end up hitting 'under', and average home team pass yards per game
condition = 'Total.isin([46.5, 47.0, 47.5]) and Home == 1'
result = 'Pts_F + Pts_A < Total'

print(f"Condition = '{condition}' and Result = '{result}' ({min(seasons)} - {max(seasons)}):\n")

cond_total = 0
res_total = 0
stat_total = 0

for season in seasons:
    season_df = nfl_df.query(f'Season == {season}')
    s_count = len(season_df)
    
    cond_df = season_df.query(condition)
    c_count = len(cond_df)

    result_df = cond_df.query(result)
    r_count = len(result_df)

    stat_count = result_df['Yds'].sum()
    
    if c_count > 0:
        print(f' {season} {r_count / c_count:.2%} ({r_count} of {c_count}) Avg Pass Yds: {stat_count/r_count:.5}')
    else:
        print('nothing')

    cond_total += c_count
    res_total += r_count
    stat_total += stat_count

print(f"\nTotal: {res_total / cond_total:.2%} ({res_total} of {cond_total}) Avg Pass Yds: {stat_total/res_total:.5}")

Condition = 'Total.isin([46.5, 47.0, 47.5]) and Home == 1' and Result = 'Pts_F + Pts_A < Total' (2016 - 2024):

 2016 50.00% (16 of 32) Avg Pass Yds: 255.44
 2017 46.88% (15 of 32) Avg Pass Yds: 216.27
 2018 60.00% (18 of 30) Avg Pass Yds: 219.5
 2019 56.25% (18 of 32) Avg Pass Yds: 219.06
 2020 63.89% (23 of 36) Avg Pass Yds: 195.26
 2021 67.57% (25 of 37) Avg Pass Yds: 208.12
 2022 57.89% (11 of 19) Avg Pass Yds: 228.36
 2023 46.67% (14 of 30) Avg Pass Yds: 210.93
 2024 35.71% (5 of 14) Avg Pass Yds: 212.4

Total: 55.34% (145 of 262) Avg Pass Yds: 216.87


In [45]:
#finding trends based on previous data
#find underdogs that cover the spread after losing by 20+ pts
prev_condition = 'Pts_A - Pts_F >= 20'
curr_condition = 'Pts_F + Pts_A < Total'
cols = ['Team', 'G']
result = 'Pts_F + Spread > Pts_A'

print(f"Previous Condition = '{prev_condition}', Current Condition = '{curr_condition}', and Result = '{result}' ({min(seasons)} - {max(seasons)}):\n")

cond_total = 0
res_total = 0

for season in seasons:
    season_df = nfl_df.query(f'Season == {season}')
    s_count = len(season_df)

    prev_cond_df = season_df.query(prev_condition)
    #create tuples of instances where previous condition was met
    prev_tuples = prev_cond_df[cols].to_records(index=False).tolist()

    #find the next game after 20+ pt loss
    curr_tuples = [(team, game+1) for team,game in prev_tuples]

    #find current df based on current tuples
    curr_df = season_df[season_df[cols].apply(func = tuple,axis=1).isin(curr_tuples)]

    cond_df = curr_df.query(curr_condition)
    c_count = len(cond_df)

    result_df = cond_df.query(result)
    r_count = len(result_df)

    if c_count > 0:
        print(f' {season} {r_count / c_count:.2%} ({r_count} of {c_count})')
    else:
        print('nothing')

    cond_total += c_count
    res_total += r_count

print(f"\nTotal: {res_total / cond_total:.2%} ({res_total} of {cond_total})")

Previous Condition = 'Pts_A - Pts_F >= 20', Current Condition = 'Pts_F + Pts_A < Total', and Result = 'Pts_F + Spread > Pts_A' (2016 - 2024):

 2016 62.50% (10 of 16)
 2017 57.14% (20 of 35)
 2018 50.00% (13 of 26)
 2019 42.11% (8 of 19)
 2020 68.18% (15 of 22)
 2021 62.86% (22 of 35)
 2022 57.14% (8 of 14)
 2023 42.86% (12 of 28)
 2024 37.50% (3 of 8)

Total: 54.68% (111 of 203)
